<h1>Model - Average Driver Pay<h1>

This notebook is modelled off the Neural Network notebook offered by University of Melbourne tutor Lucas Fern
https://github.com/lucas-fern/MAST30034-wk4-NNs

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import Dense, Normalization

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import sum,avg,max,min,mean,count
import numpy as np

In [3]:
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.driver.memory", "8g")
    .config("spark.sql.parquet.enableVectorizedReader", False)
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/08/16 16:38:14 WARN Utils: Your hostname, Sens-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.11 instead (on interface en0)
22/08/16 16:38:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/16 16:38:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sdf = spark.read.parquet('../data/curated/combined_data')

In [5]:
df = sdf.groupby('Date', 'Hour', 'PU_Location_ID').agg(avg("Temperature_C").alias("Temperature_C"), \
                                                           avg("Humidity_%").alias("Humidity_%"), \
                                                           avg("Speed_kmh").alias("Speed_kmh"), \
                                                           avg("Precip_Rate_mm").alias("Precip_rate_mm"), \
                                                           avg("Driver_pay").alias("Avg_driver_pay"), \
                                                           avg("Day_of_week").alias("Day_of_week"), \
                                                           count('Temperature_C').alias("Num_trips")).toPandas()

In [8]:
df.head(1)

,Date,Hour,PU_Location_ID,Temperature_C,Humidity_%,Speed_kmh,Precip_rate_mm,Avg_driver_pay,Day_of_week,Num_trips
0,2019-03-01,0,175,-1.11,47.0,15.29,0.0,15.38,6.0,8


In [9]:
df = df.sort_values('Date', axis = 0)
df.head(5)

,Date,Hour,PU_Location_ID,Temperature_C,Humidity_%,Speed_kmh,Precip_rate_mm,Avg_driver_pay,Day_of_week,Num_trips
728877,2019-02-01,17,42,-5.78,36.0,4.83,0.0,12.210576,6.0,330
733687,2019-02-01,9,210,-10.50,39.0,5.95,0.0,9.854417,6.0,120
733686,2019-02-01,9,263,-10.50,39.0,5.95,0.0,14.598214,6.0,280
733685,2019-02-01,7,256,-11.50,39.0,11.75,0.0,19.386034,6.0,174
1008136,2019-02-01,5,4,-10.89,39.0,13.35,0.0,16.718302,6.0,53


In [10]:
df = df.drop('Date', axis = 1)

In [11]:
df = pd.get_dummies(df, columns=['Hour', 'PU_Location_ID', 'Day_of_week'])

Make sure that model will be testing predictions on future dates by setting shuffle to false

In [22]:
TARGET_COLS = ['Avg_driver_pay']

train, test = train_test_split(df, train_size=0.8, shuffle = False)

X_train, y_train = train.drop(TARGET_COLS, axis=1).drop('Num_trips', axis=1), train[TARGET_COLS]
X_test, y_test = test.drop(TARGET_COLS, axis=1).drop('Num_trips', axis=1), test[TARGET_COLS]

In [23]:
norm_layer = Normalization()
norm_layer.adapt(X_train)

2022-08-16 18:03:33.888614: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-16 18:03:33.908803: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [26]:
model = keras.Sequential(
    [   
        norm_layer,
        Dense(10, activation='relu'),
        Dense(1, activation='relu')
    ]
)

In [27]:
model.compile(
    optimizer='adam',
    loss='MSE'
)

8 epochs chosen experimentally because this is level where validation loss starts to level out

In [29]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=16,
    validation_split=0.25,
    epochs=8
)

Epoch 1/8
81606/81606 [==============================] - ETA: 0s - loss: 13.5164

2022-08-16 18:13:11.218231: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


81606/81606 [==============================] - 363s 4ms/step - loss: 13.5164 - val_loss: 10.7629
Epoch 2/8
81606/81606 [==============================] - 361s 4ms/step - loss: 13.0173 - val_loss: 10.7994
Epoch 3/8
81606/81606 [==============================] - 362s 4ms/step - loss: 12.7904 - val_loss: 10.4236
Epoch 4/8
81606/81606 [==============================] - 361s 4ms/step - loss: 12.5140 - val_loss: 10.1190
Epoch 5/8
81606/81606 [==============================] - 361s 4ms/step - loss: 12.3817 - val_loss: 9.9785
Epoch 6/8
81606/81606 [==============================] - 364s 4ms/step - loss: 12.3218 - val_loss: 10.1366
Epoch 7/8
81606/81606 [==============================] - 363s 4ms/step - loss: 12.2615 - val_loss: 9.9495
Epoch 8/8
81606/81606 [==============================] - 358s 4ms/step - loss: 12.1783 - val_loss: 10.2112


In [38]:
comparison = y_test.iloc[:10].copy()
comparison.loc[:, 'prediction_avg_pay'] = model.predict(X_test.head(10))
comparison

1/1 [==============================] - 0s 25ms/step


,Avg_driver_pay,prediction_avg_pay
869526,35.166953,31.579432
869527,14.758911,15.270363
869528,23.911753,23.667017
869529,18.709767,16.604868
869530,13.225797,12.806939
869531,21.242042,21.144405
869532,14.571613,17.744602
869533,11.913577,11.944517
869534,11.773154,11.583035
869535,17.201562,17.291683


In [19]:
comparison = y_test.iloc[:5].copy()
comparison.loc[:, ['prediction_avg_pay','prediction_num_trips']] = model.predict(X_test.head())
comparison

1/1 [==============================] - 0s 74ms/step


2022-08-16 17:47:18.600948: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Avg_driver_pay,Num_trips,prediction_avg_pay,prediction_num_trips
869526,35.166953,489,17.625992,477.070068
869527,14.758911,257,15.374159,245.635468
869528,23.911753,154,16.240574,159.312958
869529,18.709767,172,15.935345,175.150269
869530,13.225797,69,15.106107,41.740917


In [35]:
predictions = model.predict(X_test)
errors = np.array(predictions - y_test)
squared_errors = errors**2
mean_squared_error = squared_errors.mean()

print(f'MSE: {mean_squared_error}')

13601/13601 [==============================] - 21s 2ms/step
MSE: 10.988935909237371


In [36]:
tot_sum_squares = (np.array(y_test - y_test.mean())**2).sum()
r2 = 1 - (squared_errors.sum() / tot_sum_squares)
print(f'Model R^2: {r2:.4f}')

Model R^2: 0.4530
